# CPSC 4300/6300-001 Applied Data Science (Fall 2020)

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

# CPSC4300/6300-001 Problem Set #2


# Part B: Analyzing Time Series  Data


## Introduction

The Python data science software stack have a wide range of applications in financial analysis, ranging from historical stock data analysis, correlation between search data with trends in stock prices, algorithmic trading, options modeling, portfolio risk analysis, and many others. In this notebook, you will build a dashboard for stock charting and analysis.

__Warnings:__ This intention of this notebook is to provide a hand-on exercise to develop data science skills. You shall not use the tools for actual stock trading, which involves very high risk of financial loss.

## Set up the environment

In [ ]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import mplfinance as mpf

%matplotlib inline

pd.set_option('precision', 4)

## Get The data

There are numerous sources where you get financial data. One of the most popular sources is Yahoo Finance (https://finance.yahoo.com/). If you are interested in other data sources, you may check the list at the awesome-quant github repo (https://github.com/wilsonfreitas/awesome-quant#data-sources).

To fetch data from Yahoo Finance, you can use python modules like `pandas-datareader` (https://pandas-datareader.readthedocs.io/en/latest/index.html), `yfinance` (https://aroussi.com/post/python-yahoo-finance), or others.

## Question 1. Download the historical stock prices of the Apple stock. (5 points)

1. __Write some python code to download the historical stock prices of `Apple` (stock symbol: `AAPL`) from `January, 1, 2017` to `today`. Save the data into a variable named `stock`.__

I suggest that you use the `yfinance` module since it is more convenient to get the Yahoo! Finance data than `pandas-datareader`.

You need importing all the required libraries that haven't been imported in your code.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()
stock.head(3)

In [ ]:
# test the data is of correct shape
assert stock.shape[1] == 6

# test the index is of DatetimeIndex
assert isinstance (stock.index, pd.DatetimeIndex)

In [ ]:
# test all required symbols are included 
cols = ['Open', 'High', 'Low', 'Adj Close', 'Volume']
assert all([s in stock.columns for s in cols])

In [ ]:
# test the date range in correct range
start, end, today = stock.index.min(), stock.index.max(), datetime.date.today()
assert start-pd.Timestamp(2017, 1,1) <= pd.Timedelta(days=4)
assert pd.Timestamp(today.year, today.month, today.day) - end  <= pd.Timedelta(days=4)

## Preprocess The Data

## Question 2. Calculate the simple returns using the adjusted closed price. (5 points)

Stock prices are usually non-stationary: their statistics, such as mean and variance change over time. By transforming the prices into returns, we attempt to make the time series stationary, which is a desired property in statistical modeling.

There are two types of returns: simple returns and log returns. Simple returns aggregate over assets. The simple return of a portfolio is the weighted sum of the returns of the individual assets in the portfolio. Simple returns are defined as:

$$ R_t = \frac{P_t - P_{t-1}}{P_{t-1}} = \frac{P_{t}}{P_{t-1}} - 1 $$

1. __Save the `Adj Close` price of the `AAPL` into a temporary DataFrame named `df`. Compute the simply returns of the adjusted closed price and add it to the `df` DataFrame. Name the two columns in `df` as `adj_close` and `simple_return`.__ (5 points)

Hint: you can use the `pct_change` method of a DataFrame to compute the simple returns.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()
df.head()

In [ ]:
# test the dataframe has the required columns
assert all([c in df.columns for c in ['adj_close', 'simple_return']])

In [ ]:
# verify that the first entry of the `simple_return` is of expected value 
assert np.isnan(df.simple_return[0])

In [ ]:
# verify the correct relation between adj_close and simple_return
assert all(abs(df.adj_close[i] + df.adj_close[i] * df.simple_return[i+1] - df.adj_close[i+1]) <= 1e-4 \
           for i in range(2, len(df)-5))

## Visualize Time Series

## Question 3. Plot the historical stock prices and simply returns. (5 points)

Plot the stock prices and simple returns of the Apple stock. A sample figure may look like the figure below:
![Example Stock Price Chart](https://www.palmetto.clemson.edu/dsci/figures/ps02b_aapl.png)

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Plot with the mplfinance utilties

Like `seaborn`, people have developed plotting utilities on top of `pandas` and `matplotlib` to makes it easier to create domain-specific plots. For financial plots, a popular package is `mplfinance` (https://github.com/matplotlib/mplfinance).

## Question 4. Plot the apple stock prices in the past three months. (10 points)

Use the `mplfinance` to create a plot of the past 92 days of the apple stock prices, volumes, and 10-day moving average. Your figure may look like the following figure. 
![Example MPL Finance Figure](https://www.palmetto.clemson.edu/dsci/figures/ps02b_mplfinance.png)

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Analyze Data: Identify Peaks and Troughs 

Peaks and troughs are patterns that are developed by the price action experienced by all securities. Identifying peaks and toughs is the cornerstone in stocking trend determination and technical analysis. As a general rule, higher peaks and higher troughs mean the market is going up, and will likely continue. Conversely, lower peaks and lower troughs mean the market is going down, and will likely continue.

Suppose we define a peak as an adjusted close price which is higher than both the `m` prior adjusted close prices and m succeeding adjusted close prices and a trough as the adjusted close price which is lower than itsm `m` prior and `m` succeeding adjusted close prices. Mathematically, we have the following definition:

$$peak(P_{t}) == True~iff~P_{t} >= P_{t+i}~\forall~i \in [-m, m]$$

$$trough(P_{t}) == True~iff~P_{t} <= P_{t+i}~\forall~i \in [-m, m]$$


Write some code find all the peaks and troughs of the Apple stock prices and plot those points on a stock chart figure like the one you created in Question 3. An example figure is shown as below:

![Example Figure](https://www.palmetto.clemson.edu/dsci/figures/ps02b_math_peaks.png)


### Question 5. Identify peaks and troughs (20 points)

1. __Implement the function get_troughs() to find all the troughs of the Apple stock's adjusted close price.__ (5 points)

To help you start, we have provided a naive implementation of the `get_peaks` function. 

In [ ]:
def get_peaks(s, m=10):
    """get_peaks finds the peaks of a Time Series s"""
    p = s.values
    is_peak = np.full(len(p), False)
    for i in range(m, len(p)-m-1):
        ni = range(i-m, i+m+1)
        is_peak[i] = all(p[i] >= p[ni])
    return is_peak

# YOUR CODE HERE
raise NotImplementedError()

df = stock.loc[:, ['Adj Close']]
df.rename(columns={'Adj Close':'adj_close'}, inplace=True)
df = df.assign(peak=get_peaks(df.adj_close, 10))
peaks = df[df.peak].adj_close

df = df.assign(trough=get_troughs(df.adj_close, 10))
troughs = df[df.trough].adj_close

In [ ]:
troughs
import random as rnd
rnd.seed(10)
n = rnd.randint(0, len(troughs))
date, price = troughs.index[n], troughs.values[n]
print(date, price)
print(stock[stock.index > date].head(5)['Adj Close'].values)
assert all(stock[stock.index > date].head(5)['Adj Close'].values >= price)
print(stock[stock.index < date].tail(5)['Adj Close'].values)
assert all(stock[stock.index > date].head(5)['Adj Close'].values >= price)

2. __Plot the peaks and troughs on a stock chart as the one shown above Question 5.__ (5 points)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 10), sharex=False)
# YOUR CODE HERE
raise NotImplementedError()

From the plot in Figure 5, you may notice that some of the peaks are local peaks and probably should not be counted as true peaks. For example, the peak price of 42.62 on 2019-02-06 for Apple stock is just a temporary pause of the stock price when it reached 52.02 on 2019-05-03. The same observation also goes to the troughs.

By changing the intervals of which a peak or trough is found, you may be able to remove some of the "false" peaks. But changing the intervals doesn't change the overall weakness of a simple peak identification algorithm.

One possible solution is that we introduce a set of rules for "true" peaks and troughs. For example, we may define a "true" peaks as a local peak defined in Question 5 but with the following filters:

A local peak $P_i$ is not a "true" peak if either of the following conditions meet:

   a. There is no trough between the next local peak $P_{i+j}$ and $P_{i} \lt P_{i+j}$ $\forall~j~\in [1, K-i]$ where $k$ is the total number of peaks.
   
   b. If there exists a trough $T_{k}$ between $P_{i}$ and $p_{j}$, and $P_{i} - T_{k} < \alpha P{i}$, then $T_{k}$ is not treated as a trough in a. Here $\alpha$ is a faction of price decrease, for example, $\alpha = 0.1$.   

We provide a `find_true_peaks()` function for finding the "true_peaks" and a code segment that finds the "true troughs".

In [ ]:
def find_true_peaks(peaks, troughs, alpha=0.10):
    is_true_peaks = np.full(len(peaks), False)
    for i, peak_i in enumerate(peaks):
        is_true_peak = True
        # print(i, peaks.index[i], peak_i)
        for d_j, peak_j in peaks[i+1:].items():
            # print("\t", d_j, peak_j)
            has_trough = False
            for d, trough in troughs[troughs.index > peaks.index[i]].items():
                if d > d_j:
                    break
                # print("\t\t\t", d, trough, (peak_i - trough) / peak_i)
                if peak_i - trough > alpha * peak_i:
                    has_trough = True
                    # print("\t\t\t*", d, trough, (peak_i - trough) / peak_i)
                    break
            if not has_trough and peak_i < peak_j:
                is_true_peak = False
                break
        is_true_peaks[i] = is_true_peak
    return peaks[is_true_peaks]

In [ ]:
is_true_troughs = np.full(len(troughs), False)
true_peaks = find_true_peaks(peaks, troughs, alpha=0.10)
true_troughs_dict = {}
for i, value in enumerate(true_peaks):
    dp_i, peak_i = true_peaks.index[i], value
    # find troughs between two peaks
    candidates = troughs[troughs.index > dp_i]
    if i < len(true_peaks) - 1:
        dp_j = true_peaks.index[i+1]
        candidates = candidates[candidates.index < dp_j]
    if len(candidates) > 0:
        i_min = candidates.argmin()
        true_troughs_dict[candidates.index[i_min]] = candidates[i_min]
true_troughs = pd.Series(true_troughs_dict)
true_troughs

3. __Convert the above code segment into a function `find_true_troughs()`.__ (5 points)

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Use the two functions to find the true peaks and troughs
true_peaks = find_true_peaks(peaks, troughs)
true_troughs = find_true_troughs(true_peaks, troughs)
peaks_and_troughs = true_peaks.append(true_troughs).sort_index()
df_peaks_and_troughs = pd.DataFrame({'peaks_and_troughs': peaks_and_troughs})
df_peaks_and_troughs['pct_change'] = df_peaks_and_troughs['peaks_and_troughs'].pct_change()
df_peaks_and_troughs['date'] = df_peaks_and_troughs.index
df_peaks_and_troughs['time_span'] = df_peaks_and_troughs['date'] - df_peaks_and_troughs['date'].shift(1)
df_peaks_and_troughs.drop(['date'], axis=1, inplace=True)
df_peaks_and_troughs

In [ ]:
true_peaks = find_true_peaks(peaks, troughs)
true_troughs = find_true_troughs(true_peaks, troughs)
peaks_and_troughs = true_peaks.append(true_troughs).sort_index()
assert (len(peaks_and_troughs) > 2)
if len(peaks_and_troughs) > 2:
    if peaks_and_troughs[0] > peaks_and_troughs[1]:
        assert(all([peaks_and_troughs[i] > peaks_and_troughs[i+1] for i in range(0, len(peaks_and_troughs)-2, 2)]))
    else:        
        assert(all([peaks_and_troughs[i] < peaks_and_troughs[i+1] for i in range(0, len(peaks_and_troughs)-2, 2)]))

4. __Plot the trendlines that connect true peaks and troughs on the stock price chart. Your figure may look like the one shown below.__ (5 points)

![Example Figure](https://www.palmetto.clemson.edu/dsci/figures/ps02b_true_peaks.png)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 10), sharex=False)
# YOUR CODE HERE
raise NotImplementedError()

## Question 6. Identify Outliers

When working with a dataset, you may often encounter observations that are significantly different from the majority. We can these observations outliers or anomalies. Many machine learning algorithms and statistical approaches can be influenced by outliers, leading to incorrect/biased results. Therefore, you should handle the outliers before feeding the data into a model.

For stock prices, you can identify outliers using the 3-$\sigma$ approach. In short, an observation is an outlier if its daily simple return is greater than three times the standard variance of the simple returns of $n$-day moving average. One choice of $n$ is 21.

1. __Read the code and complete the function `is_outlier()`__. (5 points)

In [ ]:
def is_outlier(row, n_sigma=3):
    """is_outlier checks if the difference between row's simple_return and row's rolliwng mean is outside the 3-sigma range."""
# YOUR CODE HERE
raise NotImplementedError()

df = stock.loc[:, ['Adj Close']]
df.rename(columns={'Adj Close':'adj_close'}, inplace=True)
df['simple_return'] = df.adj_close.pct_change()

df_rolling = df[['simple_return']].rolling(window=21).agg(['mean', 'std'])
df_rolling.columns = df_rolling.columns.droplevel()
df_outliers = df.join(df_rolling)

df_outliers['is_outliers'] = df_outliers.apply(is_outlier, axis=1)
outliers = df_outliers[df_outliers['is_outliers']]
outliers

You can visualize the outlier in a stock chart using the following code segment.

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(15, 10), sharex=True)
df.adj_close.plot(ax=ax[0])
df.simple_return.plot(ax=ax[1])
ax[0].set(title = 'AAPL Stock Prices', xlabel = 'Date', ylabel = 'Stock price ($)')
ax[0].scatter(outliers.index, outliers['adj_close'], marker='x', color='r')
ax[1].set(title = 'AAPL Simple Returns', xlabel = 'Date', ylabel = 'Simple Returns ($)')
_ =  ax[1].scatter(outliers.index, outliers['simple_return'], marker='x', color='r')

## Question 7. Technical Analysis (10 points)

Technical analysis seeks to predict price movements by examining historical data, mainly price and volume. There are many techniques in technical analysis.

In Question 4, you have created a candlestick chart, which describes the movements of a security's the open, high, low, and close prices (OHLC) for any given trading interval (seconds, minutes, hours, days, weeks, or months).

Technical analysis often uses indicator or patterns to capture signals for trading decision. One commonly used type of indicators is simple moving average, which calculates the average of a selected range of prices by the number of periods in that range.

1. __Complete the following code so that the code will plot the 20-, 50-, and 200-day moving average along with adjusted close price.__ (5 points)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 10), sharex=False)
df = pd.DataFrame(stock[['Adj Close']])
# YOUR CODE HERE
raise NotImplementedError()
df.plot(ax=ax)
ax.set(title = 'AAPL Stock Prices', xlabel = 'Date', ylabel = 'Stock price ($)')
plt.show()

2. __Based on the chart you have plot in Question 7-1, describe any pattern that may indicate the trend of price movement has reversed (i.e., from price going upward to downward, and vice versa).__ (5 points)

### Type your answer here.

## Add User Interface

After you have developed some impressive functionalities for handing stock data, you want to explore more stocks without changing the code. You also would like to ask you friends to try your first data science "product" who doesn't know much about Python gramming.

With Jupyter Widget, you can add a User Interface for your code. Below is a simple example illustrating how to add UI in a Jupyter notebook.

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
def f(x):
    print("I like", x)
    
interact(f, x=['apples','oranges']);

## Question 8. 

1. __Create a simple UI so that a user can select a stock symbol from a list of predefined stocks to plot a chart like the one you have created in Question 7 for the Apple stock.__ (15 points)

For simplicity, you can fix the date range and only allow the user to select the stock symbol from a list of symbols in which you are interested. If you are not familiar with stocks, you can set the list to be ['AAPL','TSLA', 'NVDA', 'AMD', 'WMT', 'QQQQ', 'GOOG', 'FB', 'AMZN'].

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

2. __Based on what you have learned in the assignment, which direction (UP, DOWN, FLAT, UNCLEAR) do you think the stock is likely to move in next 5 days?  You should provide a simple justification.__ (5 points)

You answer will depend on the day you work on this question. Your justification will count 3 points.

### Type your answer here.

__Congratulations! You have just completed Part B of PS02.__